# Priorisation de mails : Notebook 2

Ce notebook a pour objectif de classifier et déplacer le mails de la dernière heure
Il prend en entrée les coordonnées de la boite mails et le fichier contenant le modèle et performe la classification 

In [1]:
## Importer les librairies

import warnings
warnings.filterwarnings("ignore")
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
#Traitement de données texte
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import exchangelib
from exchangelib import DELEGATE, Account, Credentials, Configuration
from datetime import timedelta
from exchangelib import UTC_NOW
from exchangelib.folders import FolderId

import joblib
import numpy as np

import datetime
from datetime import datetime
from datetime import date

In [2]:
## extraction de text à partir du html

def html_to_text(html):
    
    soup = BeautifulSoup(html, features="html.parser")

    for script in soup(["script", "style"]):
        script.extract()  
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    return text

## définir les variables d'entrée

In [3]:

fichier_model = './Documents/Priorisation de mails/model_magasinX.joblib'
fichier_features = "./Documents/Priorisation de mails/features_magasinX.npy"

creds = Credentials(
    username="ia_mail@groupe-idea.com", 
    password="19+kT+e7!p6"
)

config = Configuration(server='outlook.office.com', credentials=creds)

account = Account(
    primary_smtp_address="n16aeroparcbis@groupe-idea.com",
    autodiscover=False, 
    config=config,
    access_type=DELEGATE
)


In [4]:
model = joblib.load(fichier_model)
features = np.load(fichier_features)

In [5]:
# dossiers[0].name

In [6]:
## Indice du dossier d'importance 

dossiers = []
idxs = []
for f in account.inbox.walk():
    dossiers.append(f.name)
    idxs.append(f.id)
    
j = dossiers.index('Mails Importants')
folder_id = idxs[j]

In [7]:
## Vous voulez traiter les mails reçus depuis quelle dates?

date_début = date(2022, 1, 1)
now = datetime.now().date()
delta = now - date_début
h = delta.days*24

In [8]:
keys = ['longeur','nombre_de_destinataires','mots_maj','lettres_maj','punct_counts','longeur_moy_mots','ratios','uniques','nb_digits','nb_ref']

for element in features :
    keys.append(str(element))
    
since = UTC_NOW() - timedelta(hours=h)
for item in account.inbox.all()\
        .filter(datetime_received__gt=since)\
        .order_by('-datetime_received'):
    
    d = dict.fromkeys(keys)
    b = html_to_text(str(item.unique_body))
    b = b.replace('\n',' ')
    text = str(item.subject) + str(item.sender.email_address) + b
    
    ### Génération des attributs textuels :


    ## Preprocessing : 

    #Preprocess text messages

    # Replace URLs with 'webaddress'
    processed = text.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                      'webaddress')

    # Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
    processed = processed.replace(r'£|\$', 'moneysymb')

    # Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
    processed = processed.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                      'phonenumbr')

    # Replace numbers with 'numbr'
    processed = processed.replace(r'\d+(\.\d+)?', 'numbr')

    # Remove punctuation
    processed = processed.replace(r'[^\w\d\s]', ' ')

    # Replace whitespace between terms with a single space
    processed = processed.replace(r'\s+', ' ')

    # Remove leading and trailing whitespace
    processed = processed.replace(r'^\s+|\s+?$', '')

    # change words to lower case 
    processed = processed.lower()

    # remove french stop words from text messages

#     stop_words = set(stopwords.words('french'))

#     processed = processed.apply(lambda x: ' '.join(
#         term for term in x.split() if term not in stop_words))

#     # remove english stop words from text messages

#     stop_words = set(stopwords.words('english'))
#     # stop_words.add('bonjour')
#     stop_words.add('please')
#     stop_words.add('mailto')
#     stop_words.add('mail')
#     stop_words.add('numbr')
#     # stop_words.add('cordialement')
#     stop_words.add('e')
#     stop_words.add('virus')
#     stop_words.add('antiphishing')
#     stop_words.add('merci')
#     stop_words.add('com')
#     stop_words.add('fr')
#     stop_words.add('https')
#     stop_words.add('bjenumbrywvybnumbrbhcmnaznumbrjvdxbllwlkzweuynumbrt')

#     processed = processed.apply(lambda x: ' '.join(
#         term for term in x.split() if term not in stop_words))

    for e in keys[10:]:
        if e in text.split():
            d[e] = True
        else:
            d[e] = False
    
    # 1 - longeur du mail

    d["longeur"] = len(b)

    # 2 - Nombre de destinataires/copies

    nb_destinataires = []
    w = str(item.sender.email_address).lower()
#         w = replace_substring(w, s, '@IDEA')  

    d['nombre_de_destinataires'] = w.count('@')

    # 3 - nombre de mots en majuscule

    maj = sum(map(str.isupper, b.split())) + sum(map(str.isupper, str(item.subject).split()))
    d['mots_maj'] = maj

    #4 - nombre de lettres majuscules

    maj = sum(i.isupper() for i in b)
    d['lettres_maj'] = maj

    # 5 -  nombre de ponctuations
    punct_counts = []

    import string 

    punct = string.punctuation

    punct_count = 0
    for element in punct :
        punct_count = punct_count + b.count(element)
        
    d['punct_counts'] = punct_count

    # # 7 -  longeur moyenne du mot

    # from statistics import mean
    
    ls = [len(x) for x in b.split()]
    lg = np.mean(ls)

    d['longueur_moy_mots'] = lg

    # # 9 -  ratio stop words

    x = sum([b.split().count(x) for x in (set(stopwords.words('french')) | set(stopwords.words('english')))])/len(b.split())

    rat = x/len(b.split())
  
    d['ratios'] = rat

    # # 10 - nombre de mots uniques

    unique = len(set(b.split()))

    d['uniques'] = unique
    
    # 11 -  nombre de digits

    nb_digits = []
    digits = list(range(10))
    m = 0
    for e in digits :
        m = m + b.count(str(m))

    d['nb_digits'] = m    

    # 12 - nombre de mots contenants digits et lettres

    import string

    x = 0
    for element in b.split() :
        if len(set(list(element))&set(list(string.ascii_lowercase)))*len(set(list(range(10)))&set(list(element))) !=0 :
            x = x + 1
    d['nb_ref'] = x
    
    ## Predecting the class of the mail
    
    classe = model.classify(d)
    print(classe)
    
    if classe == 1:
#         print(classe)
        
        l = [(item.id,item.changekey)]
        
        folder = FolderId(id = folder_id)
        account.bulk_move(l,folder,chunk_size = None)
        
          

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
